In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            images.append(img)
    return images

In [ ]:
def compute_fourier_transform(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift))
    return magnitude_spectrum

In [ ]:
def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mean = np.mean(gray)
    std_dev = np.std(gray)
    skewness = np.mean((gray - mean) ** 3) / (std_dev ** 3)
    kurtosis = np.mean((gray - mean) ** 4) / (std_dev ** 4)
    entropy = -np.sum(gray * np.log2(gray + 1e-9))
    return mean, std_dev, skewness, kurtosis, entropy

SyntaxError: unterminated string literal (detected at line 4) (<ipython-input-7-d2358b160646>, line 4)